In [17]:
import requests
import pandas as pd
from io import BytesIO
import os

In [ ]:
# https://www.dataestur.es/en/apidata/

In [24]:
# Añadimos las urls de las que necesitamos descargar archivos. Key = Name; Value = URL

downloads = {
    "FRONTUR_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/FRONTUR_DL?desde%20%28a%C3%B1o%29=2018&Pa%C3%ADs%20de%20residencia=Todos&Tipo%20de%20visitante=Todos&CCAA%20de%20destino=Todos",
    "EGATUR_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/EGATUR_DL?desde%20%28a%C3%B1o%29=2018&Pa%C3%ADs%20de%20residencia=Todos&CCAA%20de%20destino=Todos",
    "ACTIVIDADES_OCIO_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/ACTIVIDADES_OCIO_DL?CCAA=Todos&Provincia=Todos",
    "TRANSPORTE_TERRESTRE_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/TRANSPORTE_TERRESTRE_DL?desde%20%28a%C3%B1o%29=2018&Tipo%20transporte=Todos",
    "CONECTIVIDAD_AEREA_BUSQUEDAS_INICIO_VIAJE_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/CONECTIVIDAD_AEREA_BUSQUEDAS_INICIO_VIAJE_DL?Pa%C3%ADs%20origen=Todos&Ciudad%20destino=Todos&Tipo%20origen=Todos",
    "ESCUCHA_ACTIVA_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/ESCUCHA_ACTIVA_DL?Idioma=Todos&Medio=Todos&Sentimiento=Todos",
    "CST_GASTO_CONSUMO_TURISTICO_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/CST_GASTO_CONSUMO_TURISTICO_DL?Producto=Todos"
}

In [25]:
# Creamos una función para descargar un diccionario de URLS. Para ello tenemos que pasarle la url y un nombre de archivo.
# La descarga devuelve un tipo binario, así que usamos BytesIO y lo convertimos en Dataframe con Pandas.

def download_excel(url, file_name, folder="./data"):
    file_path = os.path.join(folder, f"{file_name}.xlsx")

    if os.path.exists(file_path):
        print(f"- {file_name}.xlsx already exists.")
        return pd.read_excel(file_path)

    response = requests.get(url)
    
    if response.status_code == 200:
        # Aquí hay que añadir una comprobación del tipo de contenido: si es xlsx/csv el método cambia.
        
        df = pd.read_excel(BytesIO(response.content))
        df.to_excel(file_path, index=False)
        print(f"{file_name}.xlsx downloaded and saved.")
        return df
    else:
        print(f"Failed to download {file_name}: {response.status_code}")
        return None

In [26]:
# Guardamos todos los dataframes. Llamamos nuestra función para crear un nuevo diccionario de dataframes.

datasets = {}

for name, url in downloads.items():
    df = download_excel(url, name)
    if df is not None:
        datasets[name] = df

- FRONTUR_DL.xlsx already exists.
- EGATUR_DL.xlsx already exists.
- ACTIVIDADES_OCIO_DL.xlsx already exists.
- TRANSPORTE_TERRESTRE_DL.xlsx already exists.


ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [10]:
#Para visualizar datos en conjunto:

for name, df in datasets.items():
    print(f"\n {name}")
    print("Columns:", df.columns.tolist())
    print("First 3 rows:\n", df.head(3))


 FRONTUR_DL
Columns: ['AÑO', 'MES', 'PAIS_RESIDENCIA', 'TIPO_VISITANTE', 'CCAA_DESTINO', 'VISITANTES']
First 3 rows:
     AÑO  MES PAIS_RESIDENCIA TIPO_VISITANTE CCAA_DESTINO  VISITANTES
0  2018    1        Alemania  Excursionista    Andalucía         818
1  2018    1        Alemania  Excursionista    Baleares         3694
2  2018    1        Alemania  Excursionista    Canarias        13587

 EGATUR_DL
Columns: ['AÑO', 'MES', 'PAIS_RESIDENCIA', 'TIPO_VISITANTE', 'CCAA_DESTINO', 'PERNOCTACIONES', 'GASTO_TOTAL']
First 3 rows:
     AÑO  MES PAIS_RESIDENCIA                            TIPO_VISITANTE  \
0  2018    1        Alemania  Excursionista no residente (no tránsito)   
1  2018    1        Alemania  Excursionista no residente (no tránsito)   
2  2018    1        Alemania  Excursionista no residente (no tránsito)   

  CCAA_DESTINO  PERNOCTACIONES  GASTO_TOTAL  
0    Andalucía               0    306434.84  
1     Canarias               0  14063908.66  
2     Cataluña               0   

In [48]:
# Para visualizar un archivo en concreto:

datasets["FRONTUR_DL"].tail()

,AÑO,MES,PAIS_RESIDENCIA,TIPO_VISITANTE,CCAA_DESTINO,VISITANTES
29940,2025,5,Suiza,Turista,Extremadura,87
29941,2025,5,Suiza,Turista,Galicia,2157
29942,2025,5,Suiza,Turista,País Vasco,1132
29943,2025,5,Suiza,Turista,Principado de Asturias,164
29944,2025,5,Suiza,Turista,Región de Murcia,12089


In [9]:
datasets["ACTIVIDADES_OCIO_DL"].head()

,AÑO,MES,PRODUCTO,CATEGORIA,CCAA,PROVINCIA,ENTRADAS,VISITAS_PAGINAS,GASTO_TOTAL,PRECIO_MEDIO_ENTRADA,TRANSACCIONES
0,2023,7,Espectaculos,"Cultura, teatro y danza",País Vasco,Araba/Álava,35,135,420.00,12.00,12
1,2023,7,Espectaculos,"Cultura, teatro y danza",País Vasco,Araba/Álava,2,8,6.46,3.23,1
2,2023,7,Espectaculos,Música,País Vasco,Araba/Álava,310,4171,3357.30,10.83,153
3,2023,7,Actividades,Actividades infantiles,País Vasco,Araba/Álava,39,151,207.87,5.33,17
4,2023,7,Espectaculos,"Cultura, teatro y danza",País Vasco,Araba/Álava,290,1123,2726.00,9.40,124
